In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os,cv2
from IPython.display import Image
from tqdm import tqdm,tqdm_notebook
from keras.preprocessing import image
from keras import optimizers
from keras import layers,models
from keras.applications.imagenet_utils import preprocess_input
import matplotlib.pyplot as plt
import seaborn as sns
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
print(os.listdir("../input"))

import numpy as np

In [ ]:
train_dir = "../input/train/train/"
test_dir = "../input/test/test/"
train_df = pd.read_csv('../input/train.csv')
train_df.head()

In [ ]:
Image(os.path.join("../input/train/train",train_df.iloc[0,0]),width=250,height=250)

In [ ]:
X_tr = []
Y_tr = []
imges = train_df['id'].values
for img_id in tqdm_notebook(imges):
    X_tr.append(cv2.imread(train_dir + img_id))    
    Y_tr.append(train_df[train_df['id'] == img_id]['has_cactus'].values[0])  
X_tr = np.asarray(X_tr)
X_tr = X_tr.astype('float64')
X_tr /= 255
Y_tr = np.asarray(Y_tr)

# Model Define

In [ ]:
model=models.Sequential()
model.add(layers.Conv2D(64,activation='relu',kernel_size=(1,1),input_shape=(32,32,3),padding='same',kernel_initializer='lecun_normal'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128,activation='relu',kernel_size=(1,1),padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Conv2D(256,activation='relu',kernel_size=(3,3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(units=256,activation='relu'))
model.add(layers.Dense(units=1,activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
adam=optimizers.Adam(lr=0.0001,beta_1=0.9,beta_2=0.999,epsilon=1e-8)
model.compile(optimizer=adam,loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
history = model.fit(X_tr,Y_tr,batch_size=150,epochs=20,validation_split=0.2,shuffle=True)

In [ ]:
%%time
X_tst = []
Test_imgs = []
for img_id in tqdm_notebook(os.listdir(test_dir)):
    X_tst.append(cv2.imread(test_dir + img_id))     
    Test_imgs.append(img_id)
X_tst = np.asarray(X_tst)
X_tst = X_tst.astype('float32')
X_tst /= 255

In [ ]:
X_tst

In [ ]:
y_pred=model.predict(X_tst)

In [ ]:
for i in range(len(y_pred)):
    if(y_pred[i]>0.5):y_pred[i]=1
    else:y_pred[i]=0;

In [ ]:
y_pred=np.asarray(y_pred)

In [ ]:
sub_df = pd.DataFrame(y_pred, columns=['has_cactus'])
sub_df['has_cactus'] = sub_df['has_cactus'].apply(lambda x: 1 if x > 0.75 else 0)

In [ ]:
sub_df['id'] = ''
cols = sub_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
sub_df=sub_df[cols]

In [ ]:
for i, img in enumerate(Test_imgs):
    sub_df.set_value(i,'id',img)

In [ ]:
sub_df.head()

In [ ]:
sub_df.to_csv('submission.csv',index=False)

In [ ]:
def plot_training_curves(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(1, len(acc) + 1)
    
    plt.plot(epochs, loss, 'r', label='Training loss')
    plt.plot(epochs, val_loss, 'g', label='Validation loss')
    plt.title('Losses')
    plt.xticks()
    plt.legend()
    plt.figure()
    
    plt.plot(epochs, acc, 'r', label='Training acc')
    plt.plot(epochs, val_acc, 'g', label='Validation acc')
    plt.title('Accuracies')
    plt.legend()
    plt.figure()
    
    plt.show()

In [ ]:
plot_training_curves(history)